In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from time import time
import numpy as np
import os


In [5]:
# Class for each small NN model

class MyModelA(nn.Module):
    def __init__(self):
        super(MyModelA, self).__init__()
        self.fc1 = nn.Linear(1, 4)
        self.fc2 = nn.Linear(4, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x
       
# Main Combined Model
# Class that uses all the small models combined and generated height 
class MyEnsemble(nn.Module):
    def __init__(self, modelA, modelB, modelC, modelD):
        super(MyEnsemble, self).__init__()
        
        # Models that are used to predict height from detected long bone
        self.model_lower_leg = modelA
        self.model_upper_leg = modelB
        self.model_lower_arm = modelC
        self.model_upper_arm = modelD
        
  
    # Check the measured/detected long bone according to recieved flag and reconstruct height from respective long bone
    # Multipication factors are the scaling factors used before training each network, results in original value.    
    def forward(self, x1, flag):
        if flag == 'lower_leg':
            x = self.model_lower_leg(x1)
            x*=195.5    
            
        if flag == 'upper_leg':
            x = self.model_upper_leg(x1)
            x*=204.1
            
        if flag == 'lower_arm':
            x = self.model_lower_arm(x1)
            x*=204.68
            
        if flag == 'upper_arm':
            x = self.model_upper_arm(x1)
            x*= 209.2665
            
        print('Height predicted from {} : is {} cm'.format(flag, x.item()))    
        return x


In [7]:
# Create models and load state_dicts    

# Initialising 4 models for each long bone to reconstruct height

model_lower_leg = MyModelA()
model_upper_leg = MyModelA()
model_lower_arm = MyModelA()
model_upper_arm = MyModelA()

# Paths to weights for each long bone models
PATH_lower_leg = 'Trained_models/lower_leg_pytorch_/ansur_model_2.6701'
PATH_upper_leg = 'Trained_models/upper_leg_model_pytorch_/upper_leg_5.84564208984375'
PATH_lower_arm = 'Trained_models/lower_arm_pytorch_/lower_arm_pytorch_3.489548921585083'
PATH_upper_arm = 'Trained_models/upper_arm_pytorch_/upper_arm_pytorch4.744106292724609'

# Load state dicts
model_lower_leg.load_state_dict(torch.load(PATH_lower_leg))
model_upper_leg.load_state_dict(torch.load(PATH_upper_leg))
model_lower_arm.load_state_dict(torch.load(PATH_lower_arm))
model_upper_arm.load_state_dict(torch.load(PATH_upper_arm))


#Creating the main model with all the SMALL MODELS with loaded pre-trained weights
model = MyEnsemble(model_lower_leg, model_upper_leg, model_lower_arm, model_upper_arm)




## Inference Using Knee_Height
# Test value 
lower_leg = 511.0
flag = 'lower_leg' #176.88

# Converting value to a pytorch tensor
x1 = torch.from_numpy(np.array([lower_leg], dtype='float32'))

# Printing input value in cm
print('Input value :', x1.item()/10)


# Reshaping to networks input size
x1.view((1,1))

# Passing through the main model with flag to generate height and also other segments
output = model(x1/592.0, flag)






## Inference Using Upper Leg
# Test value 
upper_leg = 44.0
flag = 'upper_leg'  # 174.8

# Converting value to a pytorch tensor
x1 = torch.from_numpy(np.array([upper_leg], dtype='float32'))

# Printing input value in cm
print('\nInput value :', x1.item())

# Reshaping to networks input size
x1.view((1,1))        

# Passing through the main model with flag to generate height and also other segments
output = model(x1/55.5, flag)






## Inference Using Sitting Height
# Test value 
lower_arm = 252.58
flag = 'lower_arm'  # 171.52

# Converting value to a pytorch tensor
x1 = torch.from_numpy(np.array([lower_arm], dtype='float32'))

# Printing input value in cm
print('\nInput value :', x1.item())

# Reshaping to networks input size
x1.view((1,1))        

# Passing through the main model with flag to generate height and also other segments
output = model(x1/280.15, flag)




## Inference Using Sitting Height Under 18
# Test value 
upper_arm = 358.96
flag = 'upper_arm'  # 182.67

# Converting value to a pytorch tensor
x1 = torch.from_numpy(np.array([upper_arm], dtype='float32'))

# Printing input value in cm
print('\nInput value :', x1.item())


# Reshaping to networks input size
x1.view((1,1))        

# Passing through the main model with flag to generate height and also other segments
output = model(x1/389.098, flag)



Input value : 51.1
Height predicted from lower_leg : is 177.85983276367188 cm

Input value : 44.0
Height predicted from upper_leg : is 176.82974243164062 cm

Input value : 252.5800018310547
Height predicted from lower_arm : is 180.19358825683594 cm

Input value : 358.9599914550781
Height predicted from upper_arm : is 181.39915466308594 cm
